In [1]:
'''Markowitz Optimization in python, using pandas, and Yahoo Finance Data. Input a 
list of tickers, and the program will calcuate historical returns, a covariance matrix, and 
finally, the individual stock weigthings that maximize the portfolio sharpe ratio, defined as
sharpe ratio = (expected portfolio return - risk free rate)/portfolio standard deviation'''



#various pandas, numpy
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats
import matplotlib.mlab as mlab
# plotting

import matplotlib.pyplot as plt

# make plots inline
%matplotlib inline

# formatting options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 10) 
pd.set_option('display.width', 82) 
pd.set_option('precision', 7)

In [2]:
'''Create the function that stores Tickers in a dataframe'''
def create_portfolio(tickers, weights=None):
    if weights is None: 
        shares = np.ones(len(tickers))/len(tickers)
    portfolio = pd.DataFrame({'Tickers': tickers, 
                              'Weights': weights}, 
                             index=tickers)
    return portfolio

In [3]:
def get_historical_closes(ticker, start_date, end_date):
    # get the data for the tickers.  This will be a panel
    p = web.DataReader(ticker, 'yahoo', start_date, end_date)    
    # convert the panel to a DataFrame and selection only Adj Close
    # while making all index levels columns
    d = p.to_frame()['Adj Close'].reset_index()
    # rename the columns
    d.rename(columns={'minor': 'Ticker', 
                      'Adj Close': 'Close'}, inplace=True)
    # pivot each ticker to a column
    pivoted = d.pivot(index='Date', columns='Ticker')
    # and drop the one level on the columns
    pivoted.columns = pivoted.columns.droplevel(0)
    return pivoted

In [13]:
closes = get_historical_closes(['MSFT', 'AAPL', 'KO', "JPM", "BRK-B", "T"], '2010-01-02', '2017-01-02')
closes[:5]

Ticker           AAPL      BRK-B        JPM         KO       MSFT          T
Date                                                                        
2010-01-04  20.696493  66.220001  35.175220  20.590786  25.031296  18.092569
2010-01-05  20.732279  66.540001  35.856571  20.341703  25.039383  18.003942
2010-01-06  20.402502  66.199997  36.053574  20.334484  24.885712  17.740501
2010-01-07  20.364788  66.459999  36.767757  20.283945  24.626907  17.541315
2010-01-08  20.500179  66.440002  36.677460  19.908524  24.796747  17.412804

In [14]:
def calc_daily_returns(closes):
    return np.log(closes/closes.shift(1))

In [15]:
daily_returns = calc_daily_returns(closes)
daily_returns[:5]

Ticker           AAPL      BRK-B        JPM         KO       MSFT          T
Date                                                                        
2010-01-04        NaN        NaN        NaN        NaN        NaN        NaN
2010-01-05  0.0017276  0.0048207  0.0191850 -0.0121706  0.0003230 -0.0049106
2010-01-06 -0.0160343 -0.0051229  0.0054792 -0.0003549 -0.0061561 -0.0147405
2010-01-07 -0.0018502  0.0039198  0.0196153 -0.0024885 -0.0104542 -0.0112913
2010-01-08  0.0066263 -0.0003009 -0.0024589 -0.0186817  0.0068728 -0.0073532

In [16]:
# calculate annual returns
def calc_annual_returns(daily_returns):
    grouped = np.exp(daily_returns.groupby(
        lambda date: date.year).sum())-1
    return grouped

In [17]:
annual_returns = calc_annual_returns(daily_returns)
annual_returns

Ticker       AAPL      BRK-B        JPM         KO       MSFT          T
2010    0.5072198  0.2097554 -0.0062917  0.2271249 -0.0794417  0.0945480
2011    0.2555803 -0.0475596 -0.1991804  0.1264312 -0.0451565  0.0902390
2012    0.3990628  0.1756225  0.3618062  0.0805128  0.0579886  0.1748772
2013    0.2583539  0.3217392  0.3673336  0.1723303  0.4429797  0.0972208
2014    0.5091870  0.2664473  0.0988281  0.0526609  0.2756461  0.0065947
2015   -0.0301371 -0.1206127  0.0837256  0.0513984  0.2269186  0.0831979
2016    0.1248042  0.2343230  0.3453638 -0.0035710  0.1507775  0.2987438

In [18]:
def calc_portfolio_var(returns, weights=None):
    if weights is None: 
        weights = np.ones(returns.columns.size) / \
        returns.columns.size
    sigma = np.cov(returns.T,ddof=0)
    var = (weights * sigma * weights.T).sum()
    return var

In [19]:
# calculate our portfolio variance (equal weighted)
calc_portfolio_var(annual_returns)

0.006837869430469324

In [20]:
def sharpe_ratio(returns, weights = None, risk_free_rate = 0.015):
    n = returns.columns.size
    if weights is None: weights = np.ones(n)/n
    # get the portfolio variance
    var = calc_portfolio_var(returns, weights)
    # and the means of the stocks in the portfolio
    means = returns.mean()
    # and return the sharpe ratio
    return (means.dot(weights) - risk_free_rate)/np.sqrt(var)

In [21]:
# calculate equal weighted sharpe ratio
sharpe_ratio(annual_returns)

1.746997894627086

In [22]:
# function to minimize
def y_f(x): return 2+x**2

In [23]:
scopt.fmin(y_f,1000)

Optimization terminated successfully.
         Current function value: 2.000000
         Iterations: 27
         Function evaluations: 54


array([0.])

In [24]:
def negative_sharpe_ratio_n_minus_1_stock(weights, 
                                          returns, 
                                          risk_free_rate):
    """
    Given n-1 weights, return a negative sharpe ratio
    """
    weights2 = sp.append(weights, 1-np.sum(weights))
    return -sharpe_ratio(returns, weights2, risk_free_rate)

In [25]:
def optimize_portfolio(returns, risk_free_rate):
    """ 
    Performs the optimization
    """
    # start with equal weights
    w0 = np.ones(returns.columns.size-1, 
                 dtype=float) * 1.0 / returns.columns.size
    # minimize the negative sharpe value
    w1 = scopt.fmin(negative_sharpe_ratio_n_minus_1_stock, 
                    w0, args=(returns, risk_free_rate))
    # build final set of weights
    final_w = sp.append(w1, 1 - np.sum(w1))
    # and calculate the final, optimized, sharpe ratio
    final_sharpe = sharpe_ratio(returns, final_w, risk_free_rate)
    return (final_w, final_sharpe)

In [26]:
# optimize our portfolio
optimize_portfolio(annual_returns, 0.015)

Optimization terminated successfully.
         Current function value: -2.529620
         Iterations: 167
         Function evaluations: 283


(array([0.13578975, 0.03538629, 0.03265717, 0.47464608, 0.0567388 ,
        0.26478191]), 2.5296195573741223)